In [1]:
import networkx as nx
import random
from networkx.algorithms.connectivity import build_auxiliary_node_connectivity
import matplotlib.pyplot as plt
from collections import Counter

# Find first match index between 2 lists
def find_match(array1, array2):
    arr = [index for index, value1, value2 in zip(range(len(array1)), array1, array2) if value1 == value2 ]
    return arr[random.randint(0,len(arr)-1)] if len(arr) > 0 else -1

# Get the key of a dictionary that corresponds to a specific value
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if(val == value):
            return key
    raise Exception("No such key!")
    
def destroy_k_vertex_disjoint_paths(G, s, t, verbose=False):
    if(not((set(s) <= G.nodes()))):
        raise Exception("Some terminals from s do not belong to graph vertices!")
    if(not((set(t) <= G.nodes()))):
        raise Exception("Some terminals from t do not belong to graph vertices!")
    # Find the auxiliary graph
    aux = build_auxiliary_node_connectivity(G)
    if(verbose):
        print("Mapping     : ",str(aux.graph['mapping']))
        print("Sources     : ",str(s))
        print("Destinations: ",str(t))
        print("************************START*****************************************")
    # Convert s, t elements to a new format of the newely generated graph
    new_s = [str(aux.graph['mapping'][vertex]) + "B" for vertex in s]
    new_t = [str(aux.graph['mapping'][vertex]) + "A" for vertex in t]
    # Initialize some needed variables
    length_s = len(s)
    length_t = len(t)
    num_paths_arr = [None] * (length_s * length_t)
    tuple_to_value = dict()
    removed_edges_from_aux = []
    # Create a dictionary that maps each pair(s, t) with an index
    for i in range(length_s):
        for j in range(length_t):
            tuple_to_value[(i,j)] = i*length_t + j
    # Initial number of edge disjoint paths between each (s, t)
    for s_index, source in enumerate(new_s):
        for t_index, destination in enumerate(new_t):
            index = tuple_to_value[(s_index, t_index)]
            try:
                num_paths_arr[index] = len(list(nx.edge_disjoint_paths(aux, source, destination))) 
            except:
                num_paths_arr[index] = None
    if(verbose):
        print("Initail edge disjoint paths is ", num_paths_arr)
    # A list to track the number of edge disjoint pathes during the execution
    inner_arr = [number_paths if(number_paths is not None) else 0 for number_paths in num_paths_arr]
    # Select one match to detemine first pair (s,t) to destroy the k-vertex disjoint paths between them
    index_match = find_match(num_paths_arr, inner_arr)
    # For loop responsible for destroying the property
    while(index_match != -1):
        # Get (s, t) from the index of match
        source, destination = get_key(tuple_to_value, index_match)
        if(verbose):
            print(f"Choosen pair (s, t) = ({s[source]}, {t[destination]})")
        # Get form (source, destiation) of G to (source, destination) of auxiliary graph
        source = new_s[source]
        destination = new_t[destination]
        # Get the paths between (s, t)
        paths = list(nx.edge_disjoint_paths(aux, source, destination))
        new_paths = []
        for path in paths:
            new_paths.append([get_key(aux.graph['mapping'], int(vertex[:-1])) for vertex in path])      
        if(verbose):
            print(f"Number of paths between {get_key(aux.graph['mapping'], int(source[:-1]))} and {get_key(aux.graph['mapping'], int(destination[:-1]))} is {len(paths)}.")
            print("And the path(s) is/are : ")
            for new_path in new_paths:
                print([new_path[0]] + new_path[1:len(new_path)-1:2] + [new_path[-1]])
        # Choose random path from the paths
        random_value = random.randint(0, len(paths) - 1)
        path = paths[random_value]
        new_path = new_paths[random_value]
        if(verbose):
            print("The choosen path to remove a vertex from it is ")
            print([new_path[0]] + new_path[1:len(new_path)-1:2] + [new_path[-1]])
        # Choose random edge that represents a vertex in the original graph
        pointer = random.choice(range(1, len(path) - 1, 2))
        removed_edges_from_aux.append((path[pointer],path[pointer+1]))
        aux.remove_edge(path[pointer],path[pointer+1])
        if(verbose):
            print("Removed vertex in this step is : ", get_key(aux.graph['mapping'],int(path[pointer][:-1])))
        # Compute the number of edge disjoint paths between each pair (s, t)
        for s_index, source in enumerate(new_s):
            for t_index, destination in enumerate(new_t):
                index = (s_index * length_t) + t_index
                try:
                    inner_arr[index] = len(list(nx.edge_disjoint_paths(aux, source, destination))) 
                except:
                    inner_arr[index] = 0
        if(verbose):
            print("After deletion of a vertex, the number of edge disjoint paths is ", inner_arr)
            print("--------------------------------------------------------------------------")
        # Test if the property is still satisfied
        index_match = find_match(num_paths_arr, inner_arr)
    # Get the vertices to remove.
    vertices = []
    for edge in removed_edges_from_aux:
        vertices.append(get_key(aux.graph['mapping'],int(edge[0][:-1])))
    return vertices

In [2]:
results = []
for i in range(100):
    if(i == 99):
        G = nx.DiGraph()
        edges = [
            (0,1),(1,2),(1,3),(1,4),(2,5),(3,5),(4,5)
        ]
        G.add_edges_from(edges)
        s = [0]
        t = [5]
        v = destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        G = nx.DiGraph()
        edges = [
            (0,1),(1,2),(1,3),(1,4),(2,5),(3,5),(4,5)
        ]
        G.add_edges_from(edges)
        s = [0]
        t = [5]
        v = destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("****************************END****************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
Sources     :  [0]
Destinations:  [5]
************************START*****************************************
Initail edge disjoint paths is  [1]
Choosen pair (s, t) = (0, 5)
Number of paths between 0 and 5 is 1.
And the path(s) is/are : 
[0, 1, 2, 5]
The choosen path to remove a vertex from it is 
[0, 1, 2, 5]
Removed vertex in this step is :  2
After deletion of a vertex, the number of edge disjoint paths is  [1]
--------------------------------------------------------------------------
Choosen pair (s, t) = (0, 5)
Number of paths between 0 and 5 is 1.
And the path(s) is/are : 
[0, 1, 3, 5]
The choosen path to remove a vertex from it is 
[0, 1, 3, 5]
Removed vertex in this step is :  1
After deletion of a vertex, the number of edge disjoint paths is  [0]
--------------------------------------------------------------------------
The resulted vertices is:  [2, 1]
****************************END****************************
The final res

In [3]:
results = []
G = nx.DiGraph()
edges = [
    (0,1),(1,2),(1,3),(1,4),(2,5),(3,5),(4,5),(4,6)
]
G.add_edges_from(edges)
s = [0]
t = [5, 6]
for i in range(100):
    if(i == 99):
        v = destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
Sources     :  [0]
Destinations:  [5, 6]
************************START*****************************************
Initail edge disjoint paths is  [1, 1]
Choosen pair (s, t) = (0, 5)
Number of paths between 0 and 5 is 1.
And the path(s) is/are : 
[0, 1, 2, 5]
The choosen path to remove a vertex from it is 
[0, 1, 2, 5]
Removed vertex in this step is :  1
After deletion of a vertex, the number of edge disjoint paths is  [0, 0]
--------------------------------------------------------------------------
The resulted vertices is:  [1]
*************************END******************************************
The final results are : 
Counter({(1,): 49, (4, 1): 19, (2, 1): 10, (2, 3, 1): 6, (4, 2, 3): 5, (2, 4, 1): 4, (2, 3, 4): 4, (4, 2, 1): 2, (2, 4, 3): 1})


In [4]:
results = []
G = nx.DiGraph()
edges = [
    (0,1),(1,5),(5,7),(0,2),(2,4),(4,7),(4,8),(9,2),(9,3),(3,6),(6,8)
]
G.add_edges_from(edges)
s = [0,9]
t = [7,8]
for i in range(100):
    if(i == 99):
        v = destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 1: 1, 5: 2, 7: 3, 2: 4, 4: 5, 8: 6, 9: 7, 3: 8, 6: 9}
Sources     :  [0, 9]
Destinations:  [7, 8]
************************START*****************************************
Initail edge disjoint paths is  [2, 1, 1, 2]
Choosen pair (s, t) = (9, 8)
Number of paths between 9 and 8 is 2.
And the path(s) is/are : 
[9, 2, 4, 8]
[9, 3, 6, 8]
The choosen path to remove a vertex from it is 
[9, 3, 6, 8]
Removed vertex in this step is :  6
After deletion of a vertex, the number of edge disjoint paths is  [2, 1, 1, 1]
--------------------------------------------------------------------------
Choosen pair (s, t) = (0, 8)
Number of paths between 0 and 8 is 1.
And the path(s) is/are : 
[0, 2, 4, 8]
The choosen path to remove a vertex from it is 
[0, 2, 4, 8]
Removed vertex in this step is :  2
After deletion of a vertex, the number of edge disjoint paths is  [1, 0, 0, 0]
--------------------------------------------------------------------------
The resulted vertices is:  [6, 2]
***

In [143]:
import networkx as nx
import random
from networkx.algorithms.connectivity import build_auxiliary_node_connectivity
import matplotlib.pyplot as plt
from collections import Counter

# Find first match index between 2 lists
def find_match(array1, array2):
    arr = [index for index, value1, value2 in zip(range(len(array1)), array1, array2) if value1 == value2 ]
    return arr[random.randint(0,len(arr)-1)] if len(arr) > 0 else -1

# Get the key of a dictionary that corresponds to a specific value
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if(val == value):
            return key
    raise Exception("No such key!")
    
def plot_graph(graph) :
    """
    This function takes a graph instance and draw it .
    """
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)
    plt.show()
    
def edited_destroy_k_vertex_disjoint_paths(G, s, t, verbose=False):
    if(not((set(s) <= G.nodes()))):
        raise Exception("Some terminals from s do not belong to graph vertices!")
    if(not((set(t) <= G.nodes()))):
        raise Exception("Some terminals from t do not belong to graph vertices!")
    # Find the auxiliary graph
    aux = build_auxiliary_node_connectivity(G)
    if(verbose):
        print("Mapping     : ",str(aux.graph['mapping']))
        print("Sources     : ",str(s))
        print("Destinations: ",str(t))
        print("************************START*****************************************")
    # Convert s, t elements to a new format of the newely generated graph
    new_s = [str(aux.graph['mapping'][vertex]) + "B" for vertex in s]
    new_t = [str(aux.graph['mapping'][vertex]) + "A" for vertex in t]
    # Initialize some needed variables
    length_s = len(s)
    length_t = len(t)
    num_paths_arr = [None] * (length_s * length_t)
    tuple_to_value = dict()
    removed_edges_from_aux = []
    frequent_vertices = ''
    # Create a dictionary that maps each pair(s, t) with an index
    for i in range(length_s):
        for j in range(length_t):
            tuple_to_value[(i,j)] = i*length_t + j
    # Initial number of edge disjoint paths between each (s, t)
    for s_index, source in enumerate(new_s):
        for t_index, destination in enumerate(new_t):
            index = tuple_to_value[(s_index, t_index)]
            try:
                paths_for_pair = list(nx.edge_disjoint_paths(aux, source, destination))
                for path in paths_for_pair:
                    for vertex in path[1:len(path)-1:2]:
                        frequent_vertices += str(get_key(aux.graph['mapping'], int(vertex[:-1])))
                        frequent_vertices += ','
                num_paths_arr[index] = len(paths_for_pair) 
            except:
                num_paths_arr[index] = None
    counter_frequent_vertices = Counter(tuple([ vertex for vertex in frequent_vertices.strip().split(',') if(vertex != '') ]))
    if(verbose):
        print(counter_frequent_vertices)
    most_common = counter_frequent_vertices.most_common()
    if(verbose):
        print("Initail edge disjoint paths is ", num_paths_arr)
        print("Most common vertices in different pair paths : ", most_common)
    # A list to track the number of edge disjoint pathes during the execution
    inner_arr = [number_paths if(number_paths is not None) else 0 for number_paths in num_paths_arr]
    index_match = find_match(num_paths_arr, inner_arr)
    # For loop responsible for destroying the property
    for index, common_vertex in enumerate(most_common):
        frequent_vertices = ''
        if(index_match == -1):
            break
        if((not(str(common_vertex[0]) in counter_frequent_vertices)) and (index != 0)):
            continue
        if(verbose):
            print(common_vertex[0])
        source_edge = str(aux.graph['mapping'][int(common_vertex[0])]) + "A"
        destination_edge = str(aux.graph['mapping'][int(common_vertex[0])]) + "B"
        if(verbose):
            print("The vertex to remove from this step is : ",int( common_vertex[0]))
            print("This vertex represents an edge : ", (source_edge, destination_edge))
        removed_edges_from_aux.append((source_edge,destination_edge))
        aux.remove_edge(source_edge,destination_edge)
        for s_index, source in enumerate(new_s):
            for t_index, destination in enumerate(new_t):
                index = (s_index * length_t) + t_index
                try:
                    paths_for_pair = list(nx.edge_disjoint_paths(aux, source, destination))
                    inner_arr[index] = len(paths_for_pair)
                    if(inner_arr[index] == num_paths_arr[index]):
                        for path in paths_for_pair:
                            for vertex in path[1:len(path)-1:2]:
                                frequent_vertices += str(get_key(aux.graph['mapping'], int(vertex[:-1])))
                                frequent_vertices += ','
                except:
                    inner_arr[index] = 0
        if(verbose):
            print("The number of edge disjoint paths between each pair is : ", inner_arr)
        counter_frequent_vertices = Counter(tuple([vertex for vertex in frequent_vertices.strip().split(',') if(vertex != '')]))
        most_common = counter_frequent_vertices.most_common()
        if(verbose):
            print("Most common vertices in different pair paths : ", most_common)
        index_match = find_match(num_paths_arr, inner_arr)
            
    #Get the vertices to remove.
    vertices = []
    for edge in removed_edges_from_aux:
        vertices.append(get_key(aux.graph['mapping'],int(edge[0][:-1])))
    return vertices

In [144]:
results = []
G = nx.DiGraph()
edges = [
    (0,1),(1,5),(5,7),(0,2),(2,4),(4,7),(4,8),(9,2),(9,3),(3,6),(6,8)
]
G.add_edges_from(edges)
s = [0,9]
t = [7,8]
for i in range(100):
    if(i == 99):
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 1: 1, 5: 2, 7: 3, 2: 4, 4: 5, 8: 6, 9: 7, 3: 8, 6: 9}
Sources     :  [0, 9]
Destinations:  [7, 8]
************************START*****************************************
Counter({'2': 4, '4': 4, '1': 1, '5': 1, '3': 1, '6': 1})
Initail edge disjoint paths is  [2, 1, 1, 2]
Most common vertices in different pair paths :  [('2', 4), ('4', 4), ('1', 1), ('5', 1), ('3', 1), ('6', 1)]
2
The vertex to remove from this step is :  2
This vertex represents an edge :  ('4A', '4B')
The number of edge disjoint paths between each pair is :  [1, 0, 0, 1]
Most common vertices in different pair paths :  []
The resulted vertices is:  [2]
*************************END******************************************
The final results are : 
Counter({(2,): 100})


In [145]:
results = []
G = nx.DiGraph()
edges = [
    (0,1),(1,5),(5,7),(0,2),(2,4),(4,7),(4,8),(9,2),(9,3),(3,6),(6,8),(5,4),(6,4)
]
G.add_edges_from(edges)
s = [0,9]
t = [7,8]
for i in range(100):
    if(i == 99):
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)
# The optimal is just to delete 4.

Mapping     :  {0: 0, 1: 1, 5: 2, 7: 3, 2: 4, 4: 5, 8: 6, 9: 7, 3: 8, 6: 9}
Sources     :  [0, 9]
Destinations:  [7, 8]
************************START*****************************************
Counter({'2': 4, '4': 4, '1': 1, '5': 1, '3': 1, '6': 1})
Initail edge disjoint paths is  [2, 1, 1, 2]
Most common vertices in different pair paths :  [('2', 4), ('4', 4), ('1', 1), ('5', 1), ('3', 1), ('6', 1)]
2
The vertex to remove from this step is :  2
This vertex represents an edge :  ('4A', '4B')
The number of edge disjoint paths between each pair is :  [1, 1, 1, 1]
Most common vertices in different pair paths :  [('4', 2), ('1', 1), ('5', 1), ('3', 1), ('6', 1)]
4
The vertex to remove from this step is :  4
This vertex represents an edge :  ('5A', '5B')
The number of edge disjoint paths between each pair is :  [1, 0, 0, 1]
Most common vertices in different pair paths :  []
The resulted vertices is:  [2, 4]
*************************END******************************************
The final resu

In [146]:
results = []
G = nx.DiGraph()
edges = [
    (0,1),(1,2),(1,3),(1,4),(2,5),(3,5),(4,5),(4,6)
]
G.add_edges_from(edges)
s = [0]
t = [5,6]
for i in range(100):
    if(i == 99):
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
Sources     :  [0]
Destinations:  [5, 6]
************************START*****************************************
Counter({'1': 2, '2': 1, '4': 1})
Initail edge disjoint paths is  [1, 1]
Most common vertices in different pair paths :  [('1', 2), ('2', 1), ('4', 1)]
1
The vertex to remove from this step is :  1
This vertex represents an edge :  ('1A', '1B')
The number of edge disjoint paths between each pair is :  [0, 0]
Most common vertices in different pair paths :  []
The resulted vertices is:  [1]
*************************END******************************************
The final results are : 
Counter({(1,): 100})


In [147]:
results = []
G = nx.DiGraph()
edges = [
    (0,2),(2,5),(5,9),(0,3),(3,6),(6,10),(1,4),(4,7),(7,9),(1,8),(8,10),(0,5),(1,7),(2,9),(3,2)
]
G.add_edges_from(edges)
s = [0,1]
t = [9,10]
for i in range(100):
    if(i == 99):
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 2: 1, 5: 2, 9: 3, 3: 4, 6: 5, 10: 6, 1: 7, 4: 8, 7: 9, 8: 10}
Sources     :  [0, 1]
Destinations:  [9, 10]
************************START*****************************************
Counter({'2': 1, '5': 1, '3': 1, '6': 1, '7': 1, '8': 1})
Initail edge disjoint paths is  [2, 1, 1, 1]
Most common vertices in different pair paths :  [('2', 1), ('5', 1), ('3', 1), ('6', 1), ('7', 1), ('8', 1)]
2
The vertex to remove from this step is :  2
This vertex represents an edge :  ('1A', '1B')
The number of edge disjoint paths between each pair is :  [1, 1, 1, 1]
Most common vertices in different pair paths :  [('3', 1), ('6', 1), ('7', 1), ('8', 1)]
3
The vertex to remove from this step is :  3
This vertex represents an edge :  ('4A', '4B')
The number of edge disjoint paths between each pair is :  [1, 0, 1, 1]
Most common vertices in different pair paths :  [('7', 1), ('8', 1)]
7
The vertex to remove from this step is :  7
This vertex represents an edge :  ('9A', '9B')
The numbe

In [148]:
results = []
G = nx.DiGraph()
edges = [
    (0,4),(0,5),(4,6),(90,6),(90,7),(5,7),(6,1),(7,2),(0,90)
]
G.add_edges_from(edges)
s = [0]
t = [1,2]
for i in range(100):
    if(i == 99):
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = edited_destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 4: 1, 5: 2, 6: 3, 90: 4, 7: 5, 1: 6, 2: 7}
Sources     :  [0]
Destinations:  [1, 2]
************************START*****************************************
Counter({'4': 1, '6': 1, '5': 1, '7': 1})
Initail edge disjoint paths is  [1, 1]
Most common vertices in different pair paths :  [('4', 1), ('6', 1), ('5', 1), ('7', 1)]
4
The vertex to remove from this step is :  4
This vertex represents an edge :  ('1A', '1B')
The number of edge disjoint paths between each pair is :  [1, 1]
Most common vertices in different pair paths :  [('90', 1), ('6', 1), ('5', 1), ('7', 1)]
6
The vertex to remove from this step is :  6
This vertex represents an edge :  ('3A', '3B')
The number of edge disjoint paths between each pair is :  [0, 1]
Most common vertices in different pair paths :  [('5', 1), ('7', 1)]
5
The vertex to remove from this step is :  5
This vertex represents an edge :  ('2A', '2B')
The number of edge disjoint paths between each pair is :  [0, 1]
Most common vertices

In [150]:
results = []
G = nx.DiGraph()
edges = [
    (0,4),(0,5),(4,6),(90,6),(90,7),(5,7),(6,1),(7,2),(0,90)
]
G.add_edges_from(edges)
s = [0]
t = [1,2]
for i in range(100):
    if(i == 99):
        v = destroy_k_vertex_disjoint_paths(G, s, t, True)
        results.append(tuple(v))
        print("The resulted vertices is: ", v)
    else:
        v = destroy_k_vertex_disjoint_paths(G, s, t, False)
        results.append(tuple(v))
print("*************************END******************************************")
print("The final results are : ")
results = Counter(tuple(results))
print(results)

Mapping     :  {0: 0, 4: 1, 5: 2, 6: 3, 90: 4, 7: 5, 1: 6, 2: 7}
Sources     :  [0]
Destinations:  [1, 2]
************************START*****************************************
Initail edge disjoint paths is  [1, 1]
Most common vertices in different pair paths :  [('4', 1), ('6', 1), ('5', 1), ('7', 1)]
The vertex to remove from this step is :  4
This vertex represents an edge :  ('1A', '1B')
The number of edge disjoint paths between each pair is :  [1, 1]
Most common vertices in different pair paths :  [('9', 1), ('0', 1), ('6', 1), ('5', 1), ('7', 1)]
The vertex to remove from this step is :  6
This vertex represents an edge :  ('3A', '3B')
The number of edge disjoint paths between each pair is :  [0, 1]
Most common vertices in different pair paths :  [('5', 1), ('7', 1)]
The vertex to remove from this step is :  5
This vertex represents an edge :  ('2A', '2B')
The number of edge disjoint paths between each pair is :  [0, 1]
Most common vertices in different pair paths :  [('9', 1), 

END.